# Data Preparation
## Step 1 : Data  Sources

In our analysis we look at data  on movie releases from IMDB, and The Numbers to answer questions for a hypothetical movie studio start-up. Our data are contained in the ../zippedData directory of this repo and will need to be unzipped and imported to be useful for this analysis. 

### Why did we choose our data?

We decided to use data from `tn.movie_budgets.csv.gz`, `imdb.title.basics.csv.gz`,`imdb.title.crew.csv.gz` and `imdb.name.basics.csv.gz`. We chose `tn.movie_budgets.csv.gz` because it provided more detailed information about revenue and production costs which allowed us to ask and answer more meaningful questions about the overall return on investment for each film. We also included `imdb.title.basics.csv.gz` in order to take a more detailed look at what _types_ of films performed best over time. Finally, we took a look at the personell files in `imdb.name.basics.csv.gz` and `imdb.title.crew.csv.gz` to answer questions about which directors were involved in successful titles.

First we will import the required packages and build an unzip function to help access our relevant files.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import gzip
from io import StringIO

%matplotlib inline

In [2]:
# This function uses gzip and io to unzip the data, two modules which were initially added to our repo by Jesus
def unzip_csv(file_location): 
    file = gzip.open(file_location, 'rb')
    content = file.read()
    file.close()
    try:
        content_str = str(content,'utf-8')
        content_data = StringIO(content_str) 
    except:
        content_str = str(content,'latin-1')
        content_data = StringIO(content_str) 
    try:
        return pd.read_csv(content_data)
    except:
        return pd.read_csv(content_data, sep='\t')

In [3]:
!ls -la ../zippedData/

total 23044
drwxr-xr-x 1 smang 197609        0 Jun 23 17:44 .
drwxr-xr-x 1 smang 197609        0 Jun 23 17:44 ..
-rw-r--r-- 1 smang 197609 18070960 Jun 23 17:44 imdb.name.basics.csv.gz
-rw-r--r-- 1 smang 197609  3459897 Jun 23 17:44 imdb.title.basics.csv.gz
-rw-r--r-- 1 smang 197609  1898523 Jun 23 17:44 imdb.title.crew.csv.gz
-rw-r--r-- 1 smang 197609   153218 Jun 23 17:44 tn.movie_budgets.csv.gz


In [4]:
#hard-coding the file-locations and nicknames into a dict for future reference
file_locations = ['../zippedData/imdb.name.basics.csv.gz'
                  ,'../zippedData/imdb.title.basics.csv.gz'
                  ,'../zippedData/tn.movie_budgets.csv.gz'
                  ,'../zippedData/imdb.title.crew.csv.gz']

file_nicknames = ['name','basics','budgets','crew']

#this dicitonary comprehension uses a zip function to smush the two lists together and then parse them into a dict
#we also have a reference for each raw df and its location on the drive.
file_dict = {k:v for k,v in zip(file_nicknames,file_locations)}

#we unzip and define frames
name= unzip_csv(file_dict['name'])
basics= unzip_csv(file_dict['basics'])
budgets= unzip_csv(file_dict['budgets'])
crew= unzip_csv(file_dict['crew'])

data_list = [
    name
    ,crew
    ,basics
    ,budgets
]

## Step 2: Cleaning the Data

In the next step we take the raw data frames and format the values to their appropriate data types, drop duplicates, null values, and redundant or irrelevant columns. We'll examine the head and info of each data frame as a starting off point:

In [5]:
for df in data_list:
    display(df.head(2))

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [6]:
#lets iterate through each data frame in the list and gather some intel about the data
for df in data_list:
    display(df.head(2),df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     146144 non-null  object
 1   directors  140417 non-null  object
 2   writers    110261 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"


...

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


None

>It looks like we have missing values in 3 of our 4 dataframes so we will have to do something about that with our data preparation. We also have a lot less information on the budgets than we do on any other file so we will preserve those records as priority. We can't make a very in depth analysis with that dataframe alone. We also want information about
_type_ of movie we're looking at, so we're going to need to match up the files by title to add genre information into the picture. Finally, we're going to need to use the crew and name data to associate the correct director to the correct movie.

Lets create a set of functions and loops that will do some of the basic cleaning for us right out of the gate. These functions will handle tasks that we want done to every dataframe in our possesion. Firstly we will want it to strip lurking whitespace out of all the object columns and column headers. Then we will look at a lambda function that drops null values for a specified set of columns so we can preserve only records that will help our analysis. We're going about it this way because the budgets df doesnt have any null values and the other three are so much more massive than that one dropping null values wont damage our pool of records very much in the end. 

In [7]:
def strip(df):
    df.columns = df.columns.str.strip()
    for column in df.columns:
        try:
            df.column = df.column.str.strip
        except:
            pass

#We'll need to get a list of the series we want formatted as money from budgets before hand.
#since its the only one with int columns we'll only develop the function this far for now.

def clean_money(df_series):
    #the map function applys the .replace to each cell in the given series, x[1:] skips the $
    return df_series.map(lambda x: int(x[1:].replace(',','')))

def dropna(dropna_dict):
    for df in dropna_dict.keys():
        print(df.values())

In [8]:
for df in data_list:
    df = strip(df)

In [9]:
#laundering the money
budgets.production_budget = clean_money(budgets.production_budget)
budgets.domestic_gross = clean_money(budgets.domestic_gross)
budgets.worldwide_gross = clean_money(budgets.worldwide_gross)

In [10]:
#We'll only drop null values in the columns that are important for our merging the columns together. 
#A graphic representation of the relationships is provided in repo as key_scheme.png in the images folder

name.drop(
    name[name.nconst.isna()==True].index
    ,inplace=True
)

crew.drop(
    crew[crew.tconst.isna()==True].index
    ,inplace=True
)


crew.drop(
    crew[crew.directors.isna()==True].index
    ,inplace=True
)

basics.drop(
    basics[basics.tconst.isna()==True].index
    ,inplace=True
)

basics.drop(
    basics[basics.genres.isna()==True].index
    ,inplace=True
)

#We're going to skip the movie column becuase we're going to merge the release year to the end of each movie title
#later to differntiate duplicate movie titles by year

for df in data_list:
    display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 32.4+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140417 entries, 0 to 146142
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     140417 non-null  object
 1   directors  140417 non-null  object
 2   writers    109008 non-null  object
dtypes: object(3)
memory usage: 4.3+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           140736 non-null  object 
 1   primary_title    140736 non-null  object 
 2   original_title   140734 non-null  object 
 3   start_year       140736 non-null  int64  
 4   runtime_minutes  112233 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 7.5+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   int64 
 5   worldwide_gross    5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


None

### Budget

In [11]:
#setting date column to datatime object for use in charts etc.
budgets.release_date = pd.to_datetime(budgets.release_date)

#adding in relevant columns
budgets['foreign_gross'] = budgets.worldwide_gross - budgets.domestic_gross
budgets['profit'] = budgets.worldwide_gross - budgets.production_budget
budgets['year'] = pd.DatetimeIndex(budgets.release_date).year
budgets['movie'] = budgets.movie+" "+budgets.year.astype(str)

#removing irrelevant columns
budgets.drop('id',axis=1,inplace=True)

budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5782 non-null   datetime64[ns]
 1   movie              5782 non-null   object        
 2   production_budget  5782 non-null   int64         
 3   domestic_gross     5782 non-null   int64         
 4   worldwide_gross    5782 non-null   int64         
 5   foreign_gross      5782 non-null   int64         
 6   profit             5782 non-null   int64         
 7   year               5782 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 361.5+ KB


In [12]:
budgets.drop_duplicates('movie',inplace=True)

In [13]:
#looks good now
budgets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5781 entries, 0 to 5781
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       5781 non-null   datetime64[ns]
 1   movie              5781 non-null   object        
 2   production_budget  5781 non-null   int64         
 3   domestic_gross     5781 non-null   int64         
 4   worldwide_gross    5781 non-null   int64         
 5   foreign_gross      5781 non-null   int64         
 6   profit             5781 non-null   int64         
 7   year               5781 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 406.5+ KB


In [14]:
budgets.head(2)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year
0,2009-12-18,Avatar 2009,425000000,760507625,2776345279,2015837654,2351345279,2009
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides 2011,410600000,241063875,1045663875,804600000,635063875,2011


### Basics
>Now the general shape of the cleaning process has been defined we can rinse and repeat on our other data sets, making them easier to use in later analysis.

In [15]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           140736 non-null  object 
 1   primary_title    140736 non-null  object 
 2   original_title   140734 non-null  object 
 3   start_year       140736 non-null  int64  
 4   runtime_minutes  112233 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 7.5+ MB


In [16]:
#reworking the title column into same format as above
basics['movie'] = basics.primary_title+" "+basics.start_year.astype(str)

#keeping only 'movie' and 'ttconst' as keys for our other data, and 'genres' for further analysis
basics.drop(['primary_title','original_title','runtime_minutes','start_year'],axis=1,inplace=True)

>The columns look correct:

In [17]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tconst  140736 non-null  object
 1   genres  140736 non-null  object
 2   movie   140736 non-null  object
dtypes: object(3)
memory usage: 4.3+ MB


In [18]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140736 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   tconst  140736 non-null  object
 1   genres  140736 non-null  object
 2   movie   140736 non-null  object
dtypes: object(3)
memory usage: 4.3+ MB


In [19]:
basics.head(2)

,tconst,genres,movie
0,tt0063540,"Action,Crime,Drama",Sunghursh 2013
1,tt0066787,"Biography,Drama",One Day Before the Rainy Season 2019


### Name

In [20]:
name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 32.4+ MB


>According to the info chart above this df contains nconst numbers and names for 600k+ people in the industry, this will help us track down the directors.

In [21]:
#dropping these since they're outside the scope of our analysis
name.drop(['primary_profession','birth_year','death_year','known_for_titles'],axis=1,inplace=True)

In [22]:
name.head(2)

,nconst,primary_name
0,nm0061671,Mary Ellen Bauder
1,nm0061865,Joseph Bauer


### Crew

In [23]:
crew.head(5)

,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943
5,tt0879859,nm2416460,NaN


In [24]:
crew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140417 entries, 0 to 146142
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     140417 non-null  object
 1   directors  140417 non-null  object
 2   writers    109008 non-null  object
dtypes: object(3)
memory usage: 4.3+ MB


In [25]:
#dropping these since they're outside the scope of our analysis
crew.drop(['writers'],axis=1,inplace=True)

In [26]:
crew.head(5)

,tconst,directors
0,tt0285252,nm0899854
2,tt0462036,nm1940585
3,tt0835418,nm0151540
4,tt0878654,"nm0089502,nm2291498,nm2292011"
5,tt0879859,nm2416460


>Since the director column is how we're planning to match up with the names df we need to get that data unnested so we can use it.

In [27]:
#this function splits the data into lists for us
crew.directors = crew.directors.map(lambda x: x.split(","))

In [28]:
crew.head()

,tconst,directors
0,tt0285252,[nm0899854]
2,tt0462036,[nm1940585]
3,tt0835418,[nm0151540]
4,tt0878654,"[nm0089502, nm2291498, nm2292011]"
5,tt0879859,[nm2416460]


In [29]:
crew.directors[4][1]

'nm2291498'

## Step 3: Merging the data
This is where things get tricky. Most of our dataframes will merge on a column with a 1-1 relationship. Either theres a matching value in the merge on column or not. Those are simple so lets handle those first

In [30]:
df = budgets.merge(basics)
df.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,tconst,genres
0,2011-05-20,Pirates of the Caribbean: On Stranger Tides 2011,410600000,241063875,1045663875,804600000,635063875,2011,tt1298650,"Action,Adventure,Fantasy"
1,2019-06-07,Dark Phoenix 2019,350000000,42762350,149762350,107000000,-200237650,2019,tt6565702,"Action,Adventure,Sci-Fi"
2,2015-05-01,Avengers: Age of Ultron 2015,330600000,459005868,1403013963,944008095,1072413963,2015,tt2395427,"Action,Adventure,Sci-Fi"
3,2018-04-27,Avengers: Infinity War 2018,300000000,678815482,2048134200,1369318718,1748134200,2018,tt4154756,"Action,Adventure,Sci-Fi"
4,2017-11-17,Justice League 2017,300000000,229024295,655945209,426920914,355945209,2017,tt0974015,"Action,Adventure,Fantasy"


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1541 entries, 0 to 1540
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1541 non-null   datetime64[ns]
 1   movie              1541 non-null   object        
 2   production_budget  1541 non-null   int64         
 3   domestic_gross     1541 non-null   int64         
 4   worldwide_gross    1541 non-null   int64         
 5   foreign_gross      1541 non-null   int64         
 6   profit             1541 non-null   int64         
 7   year               1541 non-null   int64         
 8   tconst             1541 non-null   object        
 9   genres             1541 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(3)
memory usage: 132.4+ KB


>We lose a signifigant chunk of data here but in our opinion it's a hit we have to take. We want to make an analysis of what **_types_** of movies matter, not just **_which_** ones matter so we need genre data in our final dataset.

In [32]:
name.head()

,nconst,primary_name
0,nm0061671,Mary Ellen Bauder
1,nm0061865,Joseph Bauer
2,nm0062070,Bruce Baum
3,nm0062195,Axel Baumann
4,nm0062798,Pete Baxter


>We cant do much with the name df yet since it only has the nconst numbers and names to associate to. None of those columns are in our merged df yet so we'll need to work on crew next

In [33]:
crew.head()

,tconst,directors
0,tt0285252,[nm0899854]
2,tt0462036,[nm1940585]
3,tt0835418,[nm0151540]
4,tt0878654,"[nm0089502, nm2291498, nm2292011]"
5,tt0879859,[nm2416460]


>This file will need to be merged on the tconst column. We're going to preserve our records by using a left join. We've lost quite a few already so lets see what that looks like

In [34]:
df.merge(crew,how='left').isna().sum()

release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
foreign_gross        0
profit               0
year                 0
tconst               0
genres               0
directors            4
dtype: int64

>We only lost 4 records if we do an inner join instead so let's do it since thats less than 1% of our remaining data

In [35]:
df = df.merge(crew)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 0 to 1536
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1537 non-null   datetime64[ns]
 1   movie              1537 non-null   object        
 2   production_budget  1537 non-null   int64         
 3   domestic_gross     1537 non-null   int64         
 4   worldwide_gross    1537 non-null   int64         
 5   foreign_gross      1537 non-null   int64         
 6   profit             1537 non-null   int64         
 7   year               1537 non-null   int64         
 8   tconst             1537 non-null   object        
 9   genres             1537 non-null   object        
 10  directors          1537 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(4)
memory usage: 144.1+ KB


#### Un-nesting the director data

Now let's dig into that director column. Now that each value for director has been split into a list of items we can use a pandas method to "explode" that nested data out. The new table will be full of an almost identical record of each row, with the exception of each director having their own unique value.

In [36]:
df_e = df.explode('directors')
df_e.head(5)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,tconst,genres,directors
0,2011-05-20,Pirates of the Caribbean: On Stranger Tides 2011,410600000,241063875,1045663875,804600000,635063875,2011,tt1298650,"Action,Adventure,Fantasy",nm0551128
1,2019-06-07,Dark Phoenix 2019,350000000,42762350,149762350,107000000,-200237650,2019,tt6565702,"Action,Adventure,Sci-Fi",nm1334526
2,2015-05-01,Avengers: Age of Ultron 2015,330600000,459005868,1403013963,944008095,1072413963,2015,tt2395427,"Action,Adventure,Sci-Fi",nm0923736
3,2018-04-27,Avengers: Infinity War 2018,300000000,678815482,2048134200,1369318718,1748134200,2018,tt4154756,"Action,Adventure,Sci-Fi",nm0751577
3,2018-04-27,Avengers: Infinity War 2018,300000000,678815482,2048134200,1369318718,1748134200,2018,tt4154756,"Action,Adventure,Sci-Fi",nm0751648


>Great! Now we can stitch in each name value for each corresponding **nconst** id number and then we can look at some method to compress the records back down again. Let's take a look at the merge as we did before to decide who we want to do this:

In [37]:
df_e.merge(name,how='left',left_on='directors',right_on='nconst').isna().sum()

release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
foreign_gross        0
profit               0
year                 0
tconst               0
genres               0
directors            0
nconst               0
primary_name         0
dtype: int64

>According to this there will be no null values which means we can just do a merge without specifying which type of join, <br>allowing it to default to inner.

In [38]:
df_e = df_e.merge(name,left_on='directors',right_on='nconst')
df_e.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,profit,year,tconst,genres,directors,nconst,primary_name
0,2011-05-20,Pirates of the Caribbean: On Stranger Tides 2011,410600000,241063875,1045663875,804600000,635063875,2011,tt1298650,"Action,Adventure,Fantasy",nm0551128,nm0551128,Rob Marshall
1,2018-12-19,Mary Poppins Returns 2018,130000000,171958438,341528518,169570080,211528518,2018,tt5028340,"Comedy,Family,Fantasy",nm0551128,nm0551128,Rob Marshall
2,2014-12-25,Into the Woods 2014,56200000,128002372,213116401,85114029,156916401,2014,tt2180411,"Adventure,Comedy,Drama",nm0551128,nm0551128,Rob Marshall
3,2019-06-07,Dark Phoenix 2019,350000000,42762350,149762350,107000000,-200237650,2019,tt6565702,"Action,Adventure,Sci-Fi",nm1334526,nm1334526,Simon Kinberg
4,2015-05-01,Avengers: Age of Ultron 2015,330600000,459005868,1403013963,944008095,1072413963,2015,tt2395427,"Action,Adventure,Sci-Fi",nm0923736,nm0923736,Joss Whedon


>We're not going to use this exploded dataset for much more than what we just did. Let's cut this down to just what we're interested in and forget the rest.

In [39]:
#these are the only two columns we need to get this compressed and back into our main df
df_e = df_e[['movie','primary_name','tconst']]

In [40]:
df_e.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1696 entries, 0 to 1695
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie         1696 non-null   object
 1   primary_name  1696 non-null   object
 2   tconst        1696 non-null   object
dtypes: object(3)
memory usage: 53.0+ KB


In this cell something magic happens. We group each row by movie and tconst, meaning that some of those rows now have multiple values for director again. Thats what we want though, one unique movie title tconst combo and a list of directors (sometimes there more than one apperently). So, once that series containing the directors is singled out we use an apply function to agregate a custom .join on each set of values. Then we reset the index because in this process move its set as the index. Have I lost you yet?

In [41]:
df_e = df_e.groupby(['movie','tconst'])['primary_name'].apply(", ".join).reset_index()

>Lets just suffice it to say that it works.

In [42]:
df_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1537 entries, 0 to 1536
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movie         1537 non-null   object
 1   tconst        1537 non-null   object
 2   primary_name  1537 non-null   object
dtypes: object(3)
memory usage: 36.1+ KB


>1537 records, I think thats a perfect match for our unexploded dataframe.

In [43]:
#yup
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 0 to 1536
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1537 non-null   datetime64[ns]
 1   movie              1537 non-null   object        
 2   production_budget  1537 non-null   int64         
 3   domestic_gross     1537 non-null   int64         
 4   worldwide_gross    1537 non-null   int64         
 5   foreign_gross      1537 non-null   int64         
 6   profit             1537 non-null   int64         
 7   year               1537 non-null   int64         
 8   tconst             1537 non-null   object        
 9   genres             1537 non-null   object        
 10  directors          1537 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(4)
memory usage: 144.1+ KB


In [44]:
df = df.merge(df_e,on='tconst')

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 0 to 1536
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1537 non-null   datetime64[ns]
 1   movie_x            1537 non-null   object        
 2   production_budget  1537 non-null   int64         
 3   domestic_gross     1537 non-null   int64         
 4   worldwide_gross    1537 non-null   int64         
 5   foreign_gross      1537 non-null   int64         
 6   profit             1537 non-null   int64         
 7   year               1537 non-null   int64         
 8   tconst             1537 non-null   object        
 9   genres             1537 non-null   object        
 10  directors          1537 non-null   object        
 11  movie_y            1537 non-null   object        
 12  primary_name       1537 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(6)
memory usage: 168.1+ 

In [46]:
[df.movie_x.all() == df.movie_y.all()]

[True]

>Now that we have this merge df the next step is to polish it up a bit and export the file to be used in analysis.

## Step 4: Polish and Export Data

Now the simple task of removing unneccesary columns re-ordering said columns exporting a couple files and a plethora of self pats on the back are in order. No rubber duckies were harmed in the debugging of this notebook.

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 0 to 1536
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1537 non-null   datetime64[ns]
 1   movie_x            1537 non-null   object        
 2   production_budget  1537 non-null   int64         
 3   domestic_gross     1537 non-null   int64         
 4   worldwide_gross    1537 non-null   int64         
 5   foreign_gross      1537 non-null   int64         
 6   profit             1537 non-null   int64         
 7   year               1537 non-null   int64         
 8   tconst             1537 non-null   object        
 9   genres             1537 non-null   object        
 10  directors          1537 non-null   object        
 11  movie_y            1537 non-null   object        
 12  primary_name       1537 non-null   object        
dtypes: datetime64[ns](1), int64(6), object(6)
memory usage: 168.1+ 

In [48]:
df.drop(['tconst','directors','movie_y','year'], axis=1,inplace=True)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1537 entries, 0 to 1536
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1537 non-null   datetime64[ns]
 1   movie_x            1537 non-null   object        
 2   production_budget  1537 non-null   int64         
 3   domestic_gross     1537 non-null   int64         
 4   worldwide_gross    1537 non-null   int64         
 5   foreign_gross      1537 non-null   int64         
 6   profit             1537 non-null   int64         
 7   genres             1537 non-null   object        
 8   primary_name       1537 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 120.1+ KB


In [50]:
old_names = [
    'release_date'
    ,'movie_x'                    
    ,'worldwide_gross'         
    ,'domestic_gross'                       
    ,'foreign_gross'
    ,'production_budget'     
    ,'profit'
    ,'genres'                     
    ,'primary_name'
]

new_names =[
    'release_date'
    ,'movie'
    ,'worldwide_gross'
    ,'domestic_gross'
    ,'foreign_gross'
    ,'production_budget'
    ,'profit'
    ,'genres'
    ,'director'
]

columns_mapper = {k:v for k,v in zip(old_names,new_names)}

In [51]:
df.rename(columns = columns_mapper, inplace = True)
df.reindex(columns=columns_mapper.values())

,release_date,movie,worldwide_gross,domestic_gross,foreign_gross,production_budget,profit,genres,director
0,2011-05-20,Pirates of the Caribbean: On Stranger Tides 2011,1045663875,241063875,804600000,410600000,635063875,"Action,Adventure,Fantasy",Rob Marshall
1,2019-06-07,Dark Phoenix 2019,149762350,42762350,107000000,350000000,-200237650,"Action,Adventure,Sci-Fi",Simon Kinberg
2,2015-05-01,Avengers: Age of Ultron 2015,1403013963,459005868,944008095,330600000,1072413963,"Action,Adventure,Sci-Fi",Joss Whedon
3,2018-04-27,Avengers: Infinity War 2018,2048134200,678815482,1369318718,300000000,1748134200,"Action,Adventure,Sci-Fi","Anthony Russo, Joe Russo"
4,2017-11-17,Justice League 2017,655945209,229024295,426920914,300000000,355945209,"Action,Adventure,Fantasy",Zack Snyder
...,...,...,...,...,...,...,...,...,...
1532,2017-01-27,Emily 2017,3547,3547,0,27000,-23453,Drama,Ryan Graves
1533,2015-09-01,Exeter 2015,489792,0,489792,25000,464792,"Horror,Mystery,Thriller",Marcus Nispel
1534,2015-12-01,Dutch Kills 2015,0,0,0,25000,-25000,"Crime,Drama,Thriller",Joseph Mazzella
1535,2011-11-25,The Ridges 2011,0,0,0,17300,-17300,"Drama,Horror,Thriller",Brandon Landers


In [52]:
df.drop_duplicates('movie',inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1483 entries, 0 to 1536
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   release_date       1483 non-null   datetime64[ns]
 1   movie              1483 non-null   object        
 2   production_budget  1483 non-null   int64         
 3   domestic_gross     1483 non-null   int64         
 4   worldwide_gross    1483 non-null   int64         
 5   foreign_gross      1483 non-null   int64         
 6   profit             1483 non-null   int64         
 7   genres             1483 non-null   object        
 8   director           1483 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 115.9+ KB


In [53]:
!pwd

/c/Users/smang/Documents/Flatiron/dsc-phase-1-project/Data


In [54]:
df.to_csv('data-clean.csv')